# 이커머스 데이터 분석 미션

- 첨부된 데이터는 다이어트 식품을 판매하는 어플리케이션 회사에서 나온 고객 결제 데이터 입니다. 고객들이 온라인 마케팅으로 유입된 채널에 따라, 결제를 완료한 고객도 있고, 유입이 되었으나 이탈한 고객도 있습니다. 이 데이터로 아래의 미션을 수행보세요.  
- 먼저 개인적으로 풀어본 다음, 팀별로 모여 문제를 풀어보세요.
- Chat GPT와 책, 유튜브 등 여러 자료를 활용하여 최대한 풀어보세요.
- 기한 : 2024년 5월 8일 23:59 까지
- 제출처 : woghkszhf@naver.com

### 1. 02_Data.csv파일을 불러와, df1으로 선언하고 데이터의 구조와 타입을 확인하시오.

In [1]:
import pandas as pd
import numpy as np
df1=pd.read_csv('../Data/02_Data.csv')
df1.info() ##


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회원코드        10000 non-null  object
 1   프로그램        10000 non-null  object
 2   상태          10000 non-null  object
 3   프로그램가격      10000 non-null  int64 
 4   최종결제금액      10000 non-null  int64 
 5   결제시각        10000 non-null  object
 6   결제유형        10000 non-null  object
 7   채널          10000 non-null  object
 8   보조식품가격      10000 non-null  int64 
 9   보조식품(추가구매)  10000 non-null  object
dtypes: int64(3), object(7)
memory usage: 781.4+ KB


In [31]:
df1.shape

(10000, 10)

### 2. '프로그램가격'의 평균, 최대값, 최소값을 확인하시오.

In [32]:
df1['프로그램가격'].mean()

107778.024

In [33]:
df1['프로그램가격'].max()

177000

In [34]:
df1['프로그램가격'].min()

44780

### 3. '채널'의 항목과 빈도수를 확인하고, '채널'별 '최종결제금액'의 합을 계산하여, result1.xlsx로 저장하시오.

In [35]:
df1['채널'].value_counts()

facebook     6880
others       1390
naver        1009
direct        297
email         271
google        120
instagram      33
Name: 채널, dtype: int64

In [36]:
df1.index[1000]

1000

인덱스 객체 df.index는 그 자체를 수정하여 불러올수없다, 따라서 인덱스 번호를 바꾸거나, 새롭게 짜려면, 새로운인덱스 객체를 할당하는수밖에 없다

In [37]:
df_channel_results=df1.groupby('채널',as_index=False) \
                      .agg(채널별_최종결제금액합=('최종결제금액','sum')) \
                      .sort_values('채널별_최종결제금액합',ascending=False)
                    
len=df_channel_results.shape[0]
list=[]
for i in range(0,7):  #index변환
    list.append(i+1)

df_channel_results.index=list
df_channel_results

,채널,채널별_최종결제금액합
1,facebook,825136460
2,others,149349690
3,naver,111491750
4,direct,31337160
5,email,30655080
6,google,12663480
7,instagram,3562620


결제 금액합이 크기떄문에 초기저장의 엑셀너비값으론 금액합 데이터 전체가 다 나오지않아서 -> 너비 수정

In [38]:

from openpyxl import load_workbook

df_channel_results.to_excel('result1.xlsx')

# 엑셀 파일 로드
wb = load_workbook('result1.xlsx')
ws = wb.active

# 각 열의 너비 설정 (엑셀의 컬럼명은 A,B,C,D,E,F,G,H .... 이므로)
ws.column_dimensions['B'].width = 10
ws.column_dimensions['C'].width = 25

# 수정된 내용을 새로운 파일에 저장
wb.save('result1.xlsx')
wb.close()

### 4. '채널'의 빈도수가 높은 상위 3개 항목을 뽑아, 해당 데이터의 '보조식품(추가구매)'에 따른 '최종결제금액'의 합과 평균을 계산하시오.  

값분포조사

In [39]:

df1['보조식품(추가구매)'].value_counts()

구매안함        7520
식이요법보조제A    1582
식이요법보조제B     388
단백질보충제A      340
종합비타민A       108
단백질보충제B       62
Name: 보조식품(추가구매), dtype: int64

채널의 빈도수가 가장 높은 상위 3개 항목뽑아 리스트로 저장 => query검색때 쓰려고

In [40]:
most_channel_3=df_channel_results.head(3)['채널'].to_list()
most_channel_3

['facebook', 'others', 'naver']

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Colored Text</title>
    <style>
        .red-text {
            color: red;
        }
    </style>
</head>
<body>
    <p> <span class="red-text">CheckPoint</span>.</p>
</body>
</html>

검색문 실행  
**채널의 빈도수가 가장높은 상위3개의 항목을 query로 거르고 보조식품을 groupby하는게 이 문제의 의도인가?**

case1. 채널빈도수를 검색문의 결과로 선택(select)했다고 생각했을때의 결과

In [41]:
dwf=df1.query('채널 in @most_channel_3')
dwf.groupby('채널')


In [42]:

df_chnnelB_results=dwf.groupby('보조식품(추가구매)',as_index=False) \
                      .agg(보조식품별_최종결제금액합=('최종결제금액','sum'), 보조식품별_최종결제금액평균=('최종결제금액','mean')) \
                      .sort_values('보조식품별_최종결제금액합',ascending=False) \

df=df1.copy()          
df=pd.merge(df,df_chnnelB_results,on='보조식품(추가구매)',how='left')
df.groupby('채널')
df.loc[:,['채널','보조식품(추가구매)','보조식품별_최종결제금액합','보조식품별_최종결제금액평균']]

,채널,보조식품(추가구매),보조식품별_최종결제금액합,보조식품별_최종결제금액평균
0,others,구매안함,770197460,112766.831625
1,others,구매안함,770197460,112766.831625
2,facebook,식이요법보조제A,199833090,126316.744627
3,naver,구매안함,770197460,112766.831625
4,facebook,구매안함,770197460,112766.831625
...,...,...,...,...
9995,facebook,구매안함,770197460,112766.831625
9996,facebook,식이요법보조제A,199833090,126316.744627
9997,facebook,식이요법보조제A,199833090,126316.744627
9998,facebook,구매안함,770197460,112766.831625


case2. 아니면 채널을 포함안시키는경우(빈도수 상위3개의 채널에서만 검색했다고 가정[not select 채널])

In [43]:
df_chnnelB_results=dwf.groupby('보조식품(추가구매)',as_index=False) \
                      .agg(보조식품별_최종결제금액합=('최종결제금액','sum'), 보조식품별_최종결제금액평균=('최종결제금액','mean')) \
                      .sort_values('보조식품별_최종결제금액합',ascending=False) 

df_chnnelB_results                   

,보조식품(추가구매),보조식품별_최종결제금액합,보조식품별_최종결제금액평균
0,구매안함,770197460,112766.831625
3,식이요법보조제A,199833090,126316.744627
4,식이요법보조제B,54792540,141217.886598
1,단백질보충제A,41403610,128983.208723
5,종합비타민A,12788580,133214.375000
2,단백질보충제B,6962620,112300.322581


### 5. '결제시각'데이터에서 날짜와 시간데이터를 나누어 데이터만 뽑아, '결제날짜'와 '결제시간'파생변수로 각각 선언하시오.

In [44]:
type(df1['결제시각'][0])

str

결제시각 자체가 str타입이고, 형식은 2017-05-23 오후 8:53:54.368000 의 형식이므로,   
1.최초 공백으로 문자열 split을 해서 날짜와 시간을 나누는 방법  
2.이 문자열을 datetime의 형태로 변환해서 datetime.date 로 나눠서 datetime객체의 날짜 시간 추출 메소드를 사용하는 방법 

2가지 방법이 있다.  


- 1번방법


In [45]:
df1['결제날짜']=df['결제시각'].str.split(' ').str[0] ##시리즈 타입을 문자열로 .str
df1['결제시간']=df['결제시각'].str.split(' ').str[1]+' '+df['결제시각'].str.split(' ').str[2]
df1.head()

,회원코드,프로그램,상태,프로그램가격,최종결제금액,결제시각,결제유형,채널,보조식품가격,보조식품(추가구매),결제날짜,결제시간
0,Y9RY2VSI,체중감량 프로그램 (타입B),completed,112500,112500,2017-04-14 오후 7:03:29.976000,Recurring,others,0,구매안함,2017-04-14,오후 7:03:29.976000
1,3GTN3S3B,체중감량 프로그램 (타입B),completed,44780,44780,2017-05-23 오후 8:53:54.368000,Recurring,others,0,구매안함,2017-05-23,오후 8:53:54.368000
2,6B0IG276,체중감량 프로그램 (타입 A),completed,132000,149000,2017-08-23 오후 11:39:21.840000,Recurring,facebook,17000,식이요법보조제A,2017-08-23,오후 11:39:21.840000
3,EMGRU2MO,체중감량 프로그램 (타입 A),completed,112500,112500,2017-08-28 오후 8:18:22.824000,Recurring,naver,0,구매안함,2017-08-28,오후 8:18:22.824000
4,1ELG96TX,체중감량 프로그램 (타입B),completed,44780,44780,2017-05-07 오후 5:50:30.944000,Recurring,facebook,0,구매안함,2017-05-07,오후 5:50:30.944000


- 2번방법

In [46]:
df['결제시각'].info()

<class 'pandas.core.series.Series'>
Int64Index: 10000 entries, 0 to 9999
Series name: 결제시각
Non-Null Count  Dtype 
--------------  ----- 
10000 non-null  object
dtypes: object(1)
memory usage: 156.2+ KB


결제시각 컬럼데이터를 strptime을 이용해 datetime포맷에 맞게 datetime객체타입으로 변환

In [47]:
from datetime import datetime
df['결제시각'].apply(lambda _ : datetime.strptime(_,'%Y-%m-%d %p %I:%M:%S.%f'))
df['결제시각'].info()

ValueError: time data '2017-04-14 오후 7:03:29.976000' does not match format '%Y-%m-%d %p %I:%M:%S.%f'

메소드적용해서 날짜와 시간으로 분리

In [ ]:
from datetime import datetime
df['결제날짜']=datetime.strptime(df['결제시각'].str,'%Y-%m-%d %p %I:%M:%S.%f').date()
df['결제시간']=datetime.strptime(df['결제시각'].str,'%Y-%m-%d %p %I:%M:%S.%f').time()
df.head()

TypeError: strptime() argument 1 must be str, not StringMethods

**하지만 주어진 '객체시각'데이터 문자열 포맷의 '오후','오전' 형식이 datetime.strptime 메소드에는 해당되지 않는 형식이다**  
**TypeError: strptime() argument 1 must be str, not StringMethods -> strptime은 1개의 str타입의 데이터 프레임에만 적용되는 함수이다 but 시리즈.str은 stringMethod 타입**

- &결론 pd_datetime, datetime.strptime으로 변환하기엔 맞지않는 문자열 포맷이다 

- + 추가적으로 datetime.parser 를 이용한 datetime으로 파싱하는 방법이 있는데 역시 datetime형식에 맞지 않는 데이터에는 권장되지 않는것같다.

### 6. 앞서 추출된 '결제날짜'데이터를 이용하여, '연도' / '월' / '일' 데이터를 추출하시오.

이번엔 datetime객체의 문자열포맷에 맞는 데이터를 이용하므로 datetime으로 형변환후 datetime의 객체 의 메소드를 활용해보도록 하겠다

In [49]:
import pandas as pd
df1['결제날짜'] = pd.to_datetime(df1['결제날짜'], format='%Y-%m-%d')

In [50]:
df1['결제날짜'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: 결제날짜
Non-Null Count  Dtype         
--------------  -----         
10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 78.2 KB


In [51]:
df1['결제날짜'].head()

0   2017-04-14
1   2017-05-23
2   2017-08-23
3   2017-08-28
4   2017-05-07
Name: 결제날짜, dtype: datetime64[ns]

In [52]:
df1['결제날짜'].dt.date #시리즈 타입인 df1['결제날짜'] 를 datetime객체로 이용하기 위해 .dt 로 변환후 메소드 사용

0       2017-04-14
1       2017-05-23
2       2017-08-23
3       2017-08-28
4       2017-05-07
           ...    
9995    2017-04-18
9996    2017-08-16
9997    2017-08-09
9998    2017-04-28
9999    2017-06-02
Name: 결제날짜, Length: 10000, dtype: object

In [53]:
df1['결제날짜'].dt.year 

0       2017
1       2017
2       2017
3       2017
4       2017
        ... 
9995    2017
9996    2017
9997    2017
9998    2017
9999    2017
Name: 결제날짜, Length: 10000, dtype: int64

In [55]:
df1['결제날짜'].dt.month

0       4
1       5
2       8
3       8
4       5
       ..
9995    4
9996    8
9997    8
9998    4
9999    6
Name: 결제날짜, Length: 10000, dtype: int64

In [54]:
df1['결제날짜'].dt.day

0       14
1       23
2       23
3       28
4        7
        ..
9995    18
9996    16
9997     9
9998    28
9999     2
Name: 결제날짜, Length: 10000, dtype: int64

### 7. 결제 '월'별 '최종결제금액'의 합을 계산하시오.

파생변수 선언

In [56]:
df1['결제월']=df1['결제날짜'].dt.month

In [57]:
df1['결제월'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: 결제월
Non-Null Count  Dtype
--------------  -----
10000 non-null  int64
dtypes: int64(1)
memory usage: 78.2 KB


In [59]:
final_AmountOfPayment=df1.groupby('결제월',as_index=False) \
                         .agg(최종결제금액합=('최종결제금액','sum')) \
                         .sort_values('결제월',ascending=True)

final_AmountOfPayment

,결제월,최종결제금액합
0,3,67579200
1,4,133340640
2,5,185577300
3,6,203945100
4,7,178364700
5,8,395389300


### 8. '최종결제금액'이 높은 30명의 고객 명단을 추출하여, result2.xlsx로 선언하시오.

In [66]:
df1.sort_values('최종결제금액',ascending=False,inplace=True)
high30=df1.head(30)
type(high30)
display(high30.head(5))


,회원코드,프로그램,상태,프로그램가격,최종결제금액,결제시각,결제유형,채널,보조식품가격,보조식품(추가구매),결제날짜,결제시간,결제월
4165,35OZZBQ9,체중감량 프로그램 (타입 A),completed,177000,210000,2017-06-03 오후 7:29:35.368000,Recurring,facebook,33000,식이요법보조제B,2017-06-03,오후 7:29:35.368000,6
4928,JDH0HI93,체중감량 프로그램 (타입 A),completed,177000,194000,2017-06-05 오후 1:27:24.872000,Recurring,facebook,17000,식이요법보조제A,2017-06-05,오후 1:27:24.872000,6
2736,27QV33L6,체중감량 프로그램 (타입 A),cancelled,177000,194000,2017-06-03 오후 9:49:06.408000,Recurring,facebook,0,구매안함,2017-06-03,오후 9:49:06.408000,6
3471,CR33FGED,체중감량 프로그램 (타입 A),cancelled,177000,194000,2017-06-03 오후 6:06:02.296000,Recurring,facebook,0,구매안함,2017-06-03,오후 6:06:02.296000,6
8057,D8V97HTG,체중감량 프로그램 (타입 A),completed,177000,194000,2017-06-03 오후 6:18:14.296000,Recurring,facebook,17000,식이요법보조제A,2017-06-03,오후 6:18:14.296000,6


순서대로 보기 쉽게 인덱스 번호 오름차순 설정

In [77]:
my_list = [i for i in range(1, high30.shape[0]+1)]
high30.index=my_list
high30.head()

,회원코드,프로그램,상태,프로그램가격,최종결제금액,결제시각,결제유형,채널,보조식품가격,보조식품(추가구매),결제날짜,결제시간,결제월
1,35OZZBQ9,체중감량 프로그램 (타입 A),completed,177000,210000,2017-06-03 오후 7:29:35.368000,Recurring,facebook,33000,식이요법보조제B,2017-06-03,오후 7:29:35.368000,6
2,JDH0HI93,체중감량 프로그램 (타입 A),completed,177000,194000,2017-06-05 오후 1:27:24.872000,Recurring,facebook,17000,식이요법보조제A,2017-06-05,오후 1:27:24.872000,6
3,27QV33L6,체중감량 프로그램 (타입 A),cancelled,177000,194000,2017-06-03 오후 9:49:06.408000,Recurring,facebook,0,구매안함,2017-06-03,오후 9:49:06.408000,6
4,CR33FGED,체중감량 프로그램 (타입 A),cancelled,177000,194000,2017-06-03 오후 6:06:02.296000,Recurring,facebook,0,구매안함,2017-06-03,오후 6:06:02.296000,6
5,D8V97HTG,체중감량 프로그램 (타입 A),completed,177000,194000,2017-06-03 오후 6:18:14.296000,Recurring,facebook,17000,식이요법보조제A,2017-06-03,오후 6:18:14.296000,6


1.고객데이터 전체(하나의 튜플)을 30개를 뽑을건지  
2.회원코드 컬럼데이터만 30개를 뽑을건지

In [ ]:
#1번으로 풀엇을경우

#high30.to_excel('result2.xlsx')

In [72]:
#2번으로 풀었을경우

high30_member=high30.loc[:,['회원코드']]
high30_member.head()

,회원코드
1,35OZZBQ9
2,JDH0HI93
3,27QV33L6
4,CR33FGED
5,D8V97HTG


In [75]:
from openpyxl import load_workbook

high30_member.to_excel('result2.xlsx')

# 엑셀 파일 로드
wb = load_workbook('result2.xlsx')
ws = wb.active

# 각 열의 너비 설정 (엑셀의 컬럼명은 A,B,C,D,E,F,G,H .... 이므로)
ws.column_dimensions['B'].width = 19


# 수정된 내용을 새로운 파일에 저장
wb.save('result2.xlsx')
wb.close()

### 9. '상태'가 completed에 해당하는 데이터만 뽑아, '채널'별 '최종결제금액'의 합을 계산하시오.

In [76]:
completed_channel_Payment=df1.query('상태=="completed"') \
                             .groupby('채널',as_index=False) \
                             .agg(최종결제금액합=('최종결제금액','sum')) \
                             .sort_values('최종결제금액합',ascending=False)

completed_channel_Payment

,채널,최종결제금액합
2,facebook,437954060
6,others,83813430
5,naver,63103330
0,direct,17538760
1,email,17475380
3,google,7017240
4,instagram,1890780
